# 22. 尽量使用辅助类来维护程序的状态，而不是字典和元组

动态（dynamic）的概念，指那些待保存的信息，其标识符无法提前得知（就是要保存value，但其对应的所有的key是不能提前知道的）。

现在定义一个保存学生成绩的类，使用字典：

In [1]:
class SimpleGradebook:
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = []
    
    def report_grad(self, name, score):
        self._grades[name].append(score)
        
    def average_grad(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

如果现在每个学生的成绩对应的科目我们也要保存，则就需要再嵌套一层字典了

In [2]:
class BySubjectGradebook:
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = {}
        
    def report_grade(self, name, subject, grade):
        by_subject = self._grades[name]
        grade_list = by_subject.setdefault(subject, [])  # 如果字典中有对应的key，则返回其对应的value，如果没有则将key:default
        grade_list.append(grade)                         # 添加到字典中，并返回这个default
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        total, count = 0, 0
        for grades in by_subject.values():
            total += sum(grades)
            count += len(grades)
        return total / count

以上使用起来还是简单的

但现在又有了新的需求，即每个科目的考试有多次，其中期中和期末考的比重要大，随堂考试的比较小。这时我们不再使用一系列数值组成的列表作为value，而是一系列2-tuple--`(score, weight)`作为列表的元素。

In [3]:
class WeightedGradebook:
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = {}
        
    def report_grade(self, name, subject, score, weight): # 这个编写起来还比较容易
        by_subject = self._grades[name]
        grade_list = by_subject.setdefault(subject, [])
        grade_list.append((score, weight))
        
    def average_grade(self, name):  # 但这个编写起来就比较麻烦了
        by_subject = self._grades[name]
        score_sum, score_count = 0, 0
        for subject, scores in by_subject.items():
            subject_avg, total_weight = 0, 0
            for score, weight in scores:
                # ...
        return score_sum / score_count

实际上，代码到后面已经非常难懂和繁杂了，**当嵌套多于1层的时候，就应该避免这种做法了，应去试图将其拆解为类来实现**。

## 把嵌套结构重构为类

**第一**，是每次考试的成绩，这样简单的数据不需要专门写一个类，所以依然使用tuple来进行记录

这里我们面临的问题是如果元组的长度太长，代码会变得复杂化，需要寻求更好的实现。**collections中的`namedtuple`(具名元组)**非常适合这样的工作。

In [4]:
import collections

Grade = collections.namedtuple("Grade", ("score", "weight"))

**第二**，是编写表示科目的类，其包含一系列的考试成绩。

In [6]:
class Subject:
    def __init__(self):
        self._grades = []
        
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
        
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

**第三**，编写表示学生的类，其中包含其学习的每个课程。

In [10]:
class Student:
    def __init__(self):
        self._subjects = {}
        
    def subject(self, name):
        if name not in self._subjects:
            self._subjects[name] = Subject()
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count

**最后**，编写包含所有学生成绩的容器类，其中以学生名为键，可以动态的添加学生。

In [8]:
class Gradebook:
    def __init__(self):
        self._students = {}
    
    def student(self, name):
        if name not in self._students:
            self._students[name] = Student()
        return self._students[name]

这样实现，代码量要多一些，但更加容易理解，比如

In [11]:
book = Gradebook()
albert = book.student("Albert Einstein")
math = albert.subject("math")
math.report_grade(80, 0.10)
print(albert.average_grade())

80.0


# 23. 简单的接口应该接受函数，而不是实例

**Hook**，就是在一个已有的方法上加入一些钩子，使得在该方法执行前或执行后另在做一些额外的处理。

这个技巧存在的原因：事实上如果一个项目在设计架构时考虑的足够充分，模块抽象的足够合理，设计之初为以后的扩展预留了足够的接口，那么我们完全可以不需要Hook技巧。但恰恰架构人员在项目设计之初往往没办法想的足够的深远，使得后续在扩展时深圳面临重构的痛苦，这时Hook技巧似乎可以为我们带来一记缓兵之计，通过对旧的架构进行加钩子来满足新的扩展需求。

比如`list.sort()`中有个参数`key`接受一个函数，其就可以看做是一个hook函数。

In [13]:
names = ["Socrates", "Archimedes", "Plato", "Aristotle"]
names.sort()
names

['Archimedes', 'Aristotle', 'Plato', 'Socrates']

In [14]:
names.sort(key=lambda x: len(x))
names

['Plato', 'Socrates', 'Aristotle', 'Archimedes']

**在python中，最合适hook的角色一般是函数，因为hook一般需要要任何状态，而且因为函数在python中是一个一级对象。**

我们现在对hook的功能进行探索：

第一个，我们希望创建一个字典，我们使用key去索引value的时候，如果key不存在则创建key，并将一个默认值作为其value。但是，我们还希望同时程序能够在添加新key的时候打印信息。

这个任务可以使用`defaultdict`实现。其接受到的第一个参数是一个函数，如果key不存在时的行为，返回的就是默认value值。

In [15]:
def log_missing():
    print("Key added")
    return 0

In [16]:
import collections

current = {"green": 12, "blue": 3}
increments = [
    ("red", 5),
    ("blue", 17),
    ("orange", 9)
]
result = collections.defaultdict(log_missing, current)
print("Before:", dict(result))
for key, amount in increments:
    result[key] += amount
print("After: ", dict(result))

Before: {'green': 12, 'blue': 3}
Key added
Key added
After:  {'green': 12, 'blue': 20, 'red': 5, 'orange': 9}


这里的`log_missing`函数就可以看做是一个hook，这样人我们的API更加容易构建，而且也更容易测试。

第二，我们想统计缺失的key的数量，这时候需要记录一个状态。

第一种实现的方式是使用闭包：

In [17]:
def increment_with_report(current, increments):
    added_count = 0
    
    def missing():
        nonlocal added_count
        added_count += 1
        return 0
    
    result = collections.defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount
    
    return result, added_count

In [19]:
result, count = increment_with_report(current, increments)
assert count == 2

当然，这样实现的问题是代码无比难懂。

第二种实现的方式是创建一个小型的类，把那个我们想要的状况（这里就是计数）封装到里面：

In [20]:
class CountMissing:
    def __init__(self):
        self.added = 0
        
    def missing(self):
        self.added += 1
        return 0

In [21]:
counter = CountMissing()
result = collections.defaultdict(counter.missing, current)

for key, amount in increments:
    result[key] += amount
assert counter.added == 2

一种更加明晰、合适的方法是在其中实现`__call__`方法，使得其行为和函数没有区别

In [22]:
class BetterCountMissing:
    def __init__(self):
        self.added = 0
        
    def __call__(self):
        self.added += 1
        return 0

In [23]:
counter = BetterCountMissing()
result = collections.defaultdict(counter, current)
for key, amount in increments:
    result[key] += amount
assert counter.added == 2

实际上，带状态的闭包都可以使用带有`__call__`方法的类来替代。

# 24. 以@classmethod形式的多态去通用地构建对象

**多态，指的是继承体系中的多个类都能以各自所独有的方式来实现某个方法。**这些类都满足相同的结构或继承自相同的抽象类，却有着各自不同的功能。

比如下面的MapReduce流程。

In [24]:
class InputData:
    """
    基类，表示输入的数据，有一个方法read必须要实现
    """
    def read(self):
        raise NotImplementedError

In [25]:
class PathInputData(InputData):
    """
    从磁盘中读取数据，并以字节的形式来返回待处理的数据
    """
    def __init__(self, path):
        super().__init__()
        self.path = path
        
    def read(self):
        return open(self.path).read()

我们现在可能需要许多像`PathInputData`一样的类来作为`InputData`的子类，每个子类都需要实现`read`方法，以字节的形式返回待处理的数据。

In [26]:
class Worker:
    """
    基类，MapReduce工作线程的抽象接口，接受不同的输入数据对象，但执行相同的工作
    """
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None
        
    def map(self):
        raise NotImplementedError
        
    def reduce(self, other):
        raise NotImplementedError

In [27]:
class LineCountWorder(Worker):
    """
    这是一个具体的MapReduce功能，即简单的换行符计数器。
    """
    def map(self):
        data = self.input_data.read()
        self.result = data.count("\n")
        
    def reduce(self, other):
        self.result += other.result

OK，现在我们需要考虑的问题是如何将以上构建的东西联系起来。一个基本的方法是去使用一些辅助函数来进行。

In [28]:
import os

def generate_inputs(data_dir):
    """ 读取指定路径下的所有文件，并为每个文件创建一个输入数据对象 """
    for name in os.listdir(data_dir):
        yield PathInputData(os.path.join(data_dir, name))  # 如果是其他的输入数据类，则需要重新编写一个函数，将这一句改变

In [29]:
def create_workers(input_list):
    """ 此函数接受多个输入数据对象，并创建多个换行符计数器 """
    workers = []
    for input_data in input_list:
        workers.append(LineCountWorder(input_data))  # 如果是其他的处理类，则需要重新编写一个函数，将这一句改变
    return workers

In [30]:
from threading import Thread

def execute(workers):
    """ 然后我们可以将每个workers的`map`执行都派发到多个线程中，最后反复调用`reduce`方法，将所有的结果都整合到一个 """
    threads = [Thread(target=w.map) for w in workers]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
        
    first, rest = workers[0], workers[1:]
    for worker in rest:
        first.reduce(worker)
    return first.result

In [31]:
def mapreduce(data_dir):
    inputs = generate_inputs(data_dir)
    workers = create_workers(inputs)
    return execute(workers)

我们整个过程都完成了，但我们发现了一个问题，即**我们编写的这些辅助函数不够通用，如果有其他的InputData子类，我们就需要针对性的编写其他的函数**。

其他语言中解决这个问题的方法是构造器多态，即为每个InputData子类提供特殊的构造器，使得协调MapReduce流程的那个辅助方法可以通用地构建InputData对象。但这在Python中无法做到，因为Python只允许名为`__init__`的构造器方法。

> 实际上我们有一个简单的想法来实现这个过程，即将类而不是实例作为参数输入到函数中。其实这个想法和下面要做的基于`classmethod`的实现基本上是一致的。但`classmethod`更加灵活，比如下面的实例中，我们可以直接使用类方法来生成指定类的实例的列表。

使用`@classmethod`的实现方式如下：

In [33]:
class GenericInputData:
    def read(self):
        raise NotImplementedError
        
    @classmethod
    def generate_inputs(cls, config):  # 作为一个类方法，其第一个参数必须是cls，其代表类这个对象，cls.feaure和cls.method来
        raise NotImplementedError      #  调用类属性和类方法

In [34]:
class PathInputData(GenericInputData):
    """
    从磁盘中读取数据，并以字节的形式来返回待处理的数据
    """
    def __init__(self, path):
        super().__init__()
        self.path = path
        
    def read(self):
        return open(self.path).read()
    
    @classmethod
    def generate_inputs(cls, config):          # 这个类方法生成的是该类实例组成的生成器，若是直接使用类的构造方法`__init__`，
        data_dir = config["data_dir"]          # 就需要在外面编写循环，可能不够优雅
        for name in os.listdir(data_dir):
            yield cls(os.path.join(data_dir, name))

In [35]:
class GenericWorker:
    """
    基类，MapReduce工作线程的抽象接口，接受不同的输入数据对象，但执行相同的工作
    """
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None
        
    def map(self):
        raise NotImplementedError
        
    def reduce(self, other):
        raise NotImplementedError
        
    @classmethod
    def creat_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generate_inputs(config):
            workers.append(cls(input_data))
        return workers

In [36]:
class LineCountWorder(GenericWorker):
    """
    这是一个具体的MapReduce功能，即简单的换行符计数器。
    """
    def map(self):
        data = self.input_data.read()
        self.result = data.count("\n")
        
    def reduce(self, other):
        self.result += other.result

In [38]:
def mapreduce(worker_class, input_class, config):
    workers = worker_class.create_workers(input_class, config)  # 与我们设想的直接将类作为参数输入的想法相比，其可以将内部的
    return execute(workers)                                     #  所有处理封装成一个类方法，更加合适

# 25. 使用`super`初始化父类

一种初始化父类的传统方式，是直接在子类实例化的时候调用父类的`__init__`方法。在简单的继承体系中是可行，但在多重继承时可能会出问题。

In [40]:
class MyBaseClass:
    def __init__(self, value):
        self.value = value

In [39]:
class TimesTwo:
    def __init__(self):
        self.value *= 2
        
class PlusFive:
    def __init__(self):
        self.value += 5

In [43]:
class OneWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

In [45]:
foo = OneWay(5)
foo.value # 发现其结果和`__init__`方法的执行顺序一致，但和继承的超类的顺序不一致

15

另一个问题是钻石型（菱形）继承体系，即两个父类共同来自一个公共基类。这样会使得`__init__`方法被多次执行，比如下面的例子：

In [46]:
class TimesFive(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 5
        
class PlusTwo(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 2

In [47]:
class ThisWay(TimesFive, PlusTwo):
    def __init__(self, value):
        TimesFive.__init__(self, value)  # 运行完这个，value是25
        PlusTwo.__init__(self, value)  # 但运行这个的时候，会再次调用self.value=value，所有value先变成5，再加2，所以会得到7

In [48]:
foo = ThisWay(5)
foo.value

7

为了避免上述的问题，从python2开始就加入了**方法解析顺序（MRO）机制，其规定了各个超类间的初始化顺序，使得最顶部的公共基类的`__init__`只会运行一次**。

In [49]:
class TimesFiveCorrect(MyBaseClass):
    def __init__(self, value):
        super(TimesFiveCorrect, self).__init__(value)
        self.value *= 5
        
class PlusTwoCorrect(MyBaseClass):
    def __init__(self, value):
        super(PlusTwoCorrect, self).__init__(value)
        self.value += 2

In [50]:
class GoodWay(TimesFiveCorrect, PlusTwoCorrect):
    def __init__(self, value):
        super(GoodWay, self).__init__(value)

In [51]:
foo = GoodWay(5)
foo.value

35

注意到，执行顺序可能和我们想象的并不一样。**查看正确的执行顺序，需要使用`GoodWay.mro()`类方法来查看。**

In [52]:
GoodWay.mro()  # 执行顺序和此顺序相反

[__main__.GoodWay,
 __main__.TimesFiveCorrect,
 __main__.PlusTwoCorrect,
 __main__.MyBaseClass,
 object]

但Python2中的写法有点麻烦。在Python3中，可以使用不带参数的`super`，或者`super(__class__, self)`，来应对类名改变。

In [53]:
class Explicit(MyBaseClass):
    def __init__(self, value):
        super(__class__, self).__init__(value * 2)
        
class Implicit(MyBaseClass):
    def __init__(self, value):
        super().__init__(value * 2)

In [54]:
assert Explicit(10).value == Implicit(10).value

# 26. 只在使用Mix-in组件制作工具类的时候进行多重继承

1. 首先，我们已改尽量避开多重继承。
2. 若一定要利用多重继承带来的便利和封装性，请考虑编写min-in类（一种小型的类，只定义了其他类可能需要提供的一套附加方法，但不定义自己的实例属性，也不要求使用者调用自己的`__init__`构造器）。

我们来实现一个min-in类来完成一个任务，即将内存中的对象转换为dict，便于序列化：

In [58]:
class ToDictMixin:
    def to_dict(self):
        return self._traverse_dict(self.__dict__)
    
    def _traverse_dict(self, instance_dict):
        output = {}
        for key, value in instance_dict.items():
            output[key] = self._traverse(key, value)
        return output
    
    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif isinstance(value, list):
            return [self._traverse(key, i) for i in value]
        elif hasattr(value, "__dict__"):
            return self._traverse_dict(value.__dict__)
        else:
            return value

In [59]:
class BinaryTree(ToDictMixin):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

In [61]:
tree = BinaryTree(10,
                 left=BinaryTree(7, right=BinaryTree(9)),
                 right=BinaryTree(12, left=BinaryTree(11))
                 )
tree.to_dict()

{'value': 10,
 'left': {'value': 7,
  'left': None,
  'right': {'value': 9, 'left': None, 'right': None}},
 'right': {'value': 12,
  'left': {'value': 11, 'left': None, 'right': None},
  'right': None}}

**mix-in最大的优势在于，使用者可以随时安插这些通用的功能，并在有必要的时候覆写他们。**

比如，下面定义的BinaryTree子类，拥有指向父节点的引用，则使用默认的`to_dict`处理时，会因为循环引用而陷入死循环。

In [63]:
class BinaryTreeWithParent(BinaryTree):
    def __init__(self, value, left=None, right=None, parent=None):
        super().__init__(value, left=left, right=right)
        self.parent = parent
    
    def _traverse(self, key, value):
        """
        不遍历父节点，而是只返回其值
        """
        if (isinstance(value, BinaryTreeWithParent) and key == "parent"):
            return value.value
        else:
            return super()._traverse(key, value)

In [64]:
root = BinaryTreeWithParent(10)
root.left = BinaryTreeWithParent(7, parent=root)
root.left.right = BinaryTreeWithParent(9, parent=root.left)
root.to_dict()

{'value': 10,
 'left': {'value': 7,
  'left': None,
  'right': {'value': 9, 'left': None, 'right': None, 'parent': 7},
  'parent': 10},
 'right': None,
 'parent': None}

总结一下：

* 能用mix-in组件实现的效果，就不要用多重继承来做。
* 将各个功能实现为可拔插的mix-in组件，然后令相关的类继承自己需要的那些组件，即可定制该类实例所具备的行为；
* 把简单的行为封装到mix-in组件里，然后就可以用多个mix-in组件组合出复杂的行为了。

# 27. 多有public属性，少用private属性

1. 名称前面带有两个下划线的属性为private属性；
2. 直接访问实例的private属性会报错，找不到；
3. 类方法因为还是定义在`class`代码块中的，所以可以访问到；
4. 但子类是无法访问父类的private属性的。

In [65]:
class MyObject:
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10
        
    def get_private_field(self):
        return self.__private_field

In [66]:
foo = MyObject()
foo.public_field

5

In [67]:
foo.get_private_field()

10

In [68]:
foo.__private_field

AttributeError: 'MyObject' object has no attribute '__private_field'

In [69]:
class MyOtherObject:
    def __init__(self):
        self.__private_field = 71
    
    @classmethod
    def get_private_field_of_instance(cls, instance):
        return instance.__private_field

In [70]:
bar = MyOtherObject()
MyOtherObject.get_private_field_of_instance(bar)

71

In [72]:
class MyParentObject:
    def __init__(self):
        self.__private_field = 71
        
class MyChildObject(MyParentObject):
    def get_private_field(self):
        return self.__private_field

In [73]:
baz = MyChildObject()
baz.get_private_field()

AttributeError: 'MyChildObject' object has no attribute '_MyChildObject__private_field'

**之所以我们无法访问私有属性，是因为python会对这些私有属性的名称做一些简单的变换，将类的名称放在私有属性的前面，比如`_MyParentObject__private_field`，所以实际上我们可以通过这个字段来无限制地访问到私有属性。**

In [74]:
baz._MyParentObject__private_field

71

这说明了以下的问题：

1. python无法严格保证private字段的私密性，“大家都是成年人”，更多地去考虑语言的开放性。
2. 不要盲目地去设置private属性，更多的使用protected属性（单下划线开头）即可，并在文档中将这些字段的合理用法告知子类的开发者。
3. 当子类不受自己控制的时候，可以考虑使用private属性去避免名称冲突。

# 28. 继承`collections.abc`来实现自定义的容器类型

定义一个自定义的列表类型，统计各元素出现频率的方法

首先的想法是，直接继承`list`

In [76]:
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members)
    
    def frequency(self):
        counts = {}
        for item in self:
            counts.setdefault(item, 0)
            counts[item] += 1
        return counts

In [77]:
foo = FrequencyList(["a", "b", "a", "c", "b", "a", "d"])
print("Length is", len(foo))
print("Frequency before pop:", foo.frequency())
foo.pop()
print("After pop:", repr(foo))
print("Frequency after pop:", foo.frequency())

Length is 7
Frequency before pop: {'a': 3, 'b': 2, 'c': 1, 'd': 1}
After pop: ['a', 'b', 'a', 'c', 'b', 'a']
Frequency after pop: {'a': 3, 'b': 2, 'c': 1}


因为和`list`差距太大，现在我们无法直接继承`list`，而是必须从头开始来构建。而且我们还需要能够有列表的一系列行为，比如`len`、索引、切片等。

这些行为都可以通过实现一些特殊的方法来添加，比如：

* `__getitem__`方法实现索引和切片（配合`slice`输出的切片对象）；
* `__len__`方法实现了`len`长度；
* ...

但一个一个实现太麻烦了，有比较简单的方法吗？

**collections.abc模块定义了一系列的抽象基类，其提供了每一种容器类型应具备的常用方法，如果其子类忘记实现了某个方法，则其会报错。而如果实现其提示的必要方法后，其他的一些基于此的方法也会同时实现。**

In [78]:
from collections.abc import Sequence

class BadType(Sequence):
    pass

foo = BadType()

TypeError: Can't instantiate abstract class BadType with abstract methods __getitem__, __len__

In [81]:
class BinaryNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
        
class IndexableNode(BinaryNode):
    def _search(self, source, index, read_left):
        # DFS
        # return (found, count)
        pass
            
    def __getitem__(self, index):
        found, _ = self._search(0, index)
        if not found:
            raise IndexError("Index out of range")
        return found.value
    
    def __len__(self):
        _, count = self._search(0, None)
        return count

class SequenceNode(IndexableNode, Sequence):  # 这些基类也有点想mix-in组件
    pass